imports

In [6]:
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import structcol as sc
import structcol.refractive_index as ri
from structcol import montecarlo as mc
import event_distribution as ed

Properties of system

In [34]:
ntrajectories = 30000       # number of trajectories, usually 65000
ntraj_save = 3000
nevents = 300                           # number of scattering events in each trajectory
seed = None
wavelengths = sc.Quantity(np.arange(400., 810, 10),'nm') #sc.Quantity(np.array([550]),'nm') 
particle_radius = sc.Quantity('140 nm') #sc.Quantity('140 nm')
volume_fraction = sc.Quantity(0.56, '') #sc.Quantity(0.56, '')
n_i = sc.Quantity(np.linspace(2.1e-4j, 0j,wavelengths.size),'')
n_particle = ri.n('polystyrene', wavelengths) + n_i#sc.Quantity(1.54, '')      # refractive indices can be specified as pint quantities or
n_matrix = ri.n('vacuum', wavelengths)      # called from the refractive_index module. n_matrix is the 
n_medium = ri.n('vacuum', wavelengths)      # space within sample. n_medium is outside the sample.
z_low = sc.Quantity('0.0 um')
thickness = sc.Quantity('10 um') #110
particle = 'ps'
matrix = 'air'

### Find event distribution of reflectance for a range of wavelengths 
(for film geometry)

In [36]:
t0_tot = time.time()

refl_events = np.zeros((wavelengths.size,2*nevents+1))
refl_events_fresnel_samp = np.zeros((wavelengths.size,2*nevents+1))
refl_events_fresnel_avg = np.zeros((wavelengths.size,2*nevents+1))
reflectance = np.zeros(wavelengths.size)
p = np.zeros((wavelengths.size,200))
lscat = np.zeros(wavelengths.size)
zpos = np.zeros((wavelengths.size, nevents+1, ntraj_save))
kz = np.zeros((wavelengths.size, nevents, ntraj_save))
theta_event_traj = np.zeros((wavelengths.size, nevents, ntraj_save))
tir_single = np.zeros((wavelengths.size))
tir_single_refl = np.zeros((wavelengths.size))
tir_all = np.zeros((wavelengths.size))
tir_all_refl = np.zeros((wavelengths.size))
tir_indices_single = np.zeros((wavelengths.size, ntrajectories))

for i in range(wavelengths.size):
    t0 = time.time()
    print(wavelengths[i])
    
    n_sample = ri.n_eff(n_particle[i], n_matrix[i], volume_fraction)
    #print(180-180/np.pi*np.arcsin(1/n_sample))
    
    # Calculate the phase function and scattering and absorption coefficients from the single scattering model
    # (this absorption coefficient is of the scatterer, not of an absorber added to the system)
    p[i,:], mu_scat, mu_abs = mc.calc_scat(particle_radius, n_particle[i], n_sample, volume_fraction, wavelengths[i])
    lscat[i] = 1/mu_scat.magnitude # microns
    
    # Initialize the trajectories
    r0, k0, W0 = mc.initialize(nevents, ntrajectories, n_medium[i], n_sample, incidence_angle = 0, seed = seed)
    r0 = sc.Quantity(r0, 'um')
    k0 = sc.Quantity(k0, '')
    W0 = sc.Quantity(W0, '')
    
    ###############################################################################
    # Generate a matrix of all the randomly sampled angles first 
    sintheta, costheta, sinphi, cosphi, theta, _ = mc.sample_angles(nevents, ntrajectories, p[i,:])
    sintheta = np.sin(theta)
    costheta = np.cos(theta)
    
    # Create step size distribution
    step = mc.sample_step(nevents, ntrajectories, mu_abs, mu_scat)
        
    # Create trajectories object
    trajectories = mc.Trajectory(r0, k0, W0)
    
    # Run photons
    trajectories.absorb(mu_abs, step)                         
    trajectories.scatter(sintheta, costheta, sinphi, cosphi)         
    trajectories.move(step)
    
    ################### Calculate reflection and transmition   
    refl_indices, trans_indices,\
    inc_refl_per_traj, refl_per_traj, trans_per_traj,\
    trans_frac, refl_frac,\
    refl_fresnel, trans_fresnel, reflectance[i], tir_refl_bool = mc.calc_refl_trans(trajectories, z_low, thickness, 
                                                                             n_medium[i], n_sample, 
                                                                             return_extra = True)


    ################### Calculate event distribution ##########################
    
    refl_events[i,:], trans_events = ed.calc_refl_trans_event(refl_per_traj, inc_refl_per_traj, trans_per_traj, 
                              refl_indices, trans_indices, nevents)
 
    # scattering angle
    #theta_event_traj[i,1:,:] = theta[0:-1,:]#ed.calc_thetas_event_traj(theta, refl_indices, nevents, ntraj = ntraj_save)
    
    # total internal reflection
    tir_all[i],\
    tir_all_refl[i],\
    tir_single[i],\
    tir_single_refl[i],\
    tir_indices_single[i,:] = ed.calc_tir(tir_refl_bool, refl_indices, trans_indices, inc_refl_per_traj, 
                                     trajectories.weight, 
                                     trajectories.direction[2,:,:],
                                     ntrajectories, 
                                     n_sample, 
                                     n_medium[i])

    # fresnel
#    pdf_refl, pdf_trans =  ed.calc_pdf_scat(refl_events[i,:], trans_events, nevents)
#    
#    refl_events_fresnel_avg[i,:] = ed.calc_refl_event_fresnel_avg(refl_events[i,:], refl_indices, trans_indices, 
#                            refl_fresnel, trans_fresnel,
#                            refl_frac, trans_frac, nevents)
#    
#    
#    refl_events_fresnel_samp[i,:] = ed.calc_refl_event_fresnel_pdf(refl_events[i,:], pdf_refl, pdf_trans, refl_indices, 
#                            refl_fresnel, trans_fresnel,
#                            refl_frac, trans_frac, nevents)

    # information about trajectories we want to save
    exit_indices = refl_indices + trans_indices
    #for tr in range(0,ntraj_save): # loop thru all trajectories
    #    zpos[i, 0:int(exit_indices[tr])+1,tr] = trajectories.position[2, 0:int(exit_indices[tr]) + 1, tr]
    #    kz[i, 0:int(exit_indices[tr]),tr] = trajectories.direction[2,0:int(exit_indices[tr]), tr]
    #kz[i, :, :] = trajectories.direction[2,:,:]
    
    # keep track of time to run
    t1 = time.time()
    t_wl = t1-t0
    print('time: ' + str(t_wl/60) + ' min')
    
t_tot = time.time()-t0_tot
print('total time: ' + str(t_tot/3600) + ' hr')

400.0 nanometer
time: 0.08486729462941488 min
410.0 nanometer
time: 0.08329146305720012 min
420.0 nanometer
time: 0.08341760238011678 min
430.0 nanometer
time: 0.08401244878768921 min
440.0 nanometer
time: 0.08379432757695517 min
450.0 nanometer
time: 0.08430116573969523 min
460.0 nanometer
time: 0.08496365944544475 min
470.0 nanometer
time: 0.08493456840515137 min
480.0 nanometer
time: 0.0850528359413147 min
490.0 nanometer
time: 0.08517729043960572 min
500.0 nanometer
time: 0.08529622157414754 min
510.0 nanometer
time: 0.08482808272043864 min
520.0 nanometer
time: 0.08471677700678508 min
530.0 nanometer
time: 0.08458322286605835 min
540.0 nanometer
time: 0.0848677396774292 min
550.0 nanometer
time: 0.08434693813323975 min
560.0 nanometer
time: 0.08433937231699626 min
570.0 nanometer
time: 0.08464368184407552 min
580.0 nanometer
time: 0.08471204042434692 min
590.0 nanometer
time: 0.08452279170354207 min
600.0 nanometer
time: 0.08684895833333334 min
610.0 nanometer
time: 0.086987058321

##### Save results

In [37]:
ed.save_data(particle,
             matrix,
             particle_radius,
             volume_fraction,
             thickness,
             reflectance,
             refl_events,
             wavelengths, 
             nevents, 
             ntrajectories,
             theta_event_traj = theta_event_traj,
             tir_all = tir_all, 
             tir_all_refl = tir_all_refl,
             tir_single = tir_single,
             tir_single_refl = tir_single_refl,
             tir_indices_single = tir_indices_single)

### Find event distribution of reflectance for a range of scattering angles (for a single wavelength)
(for film geometry)

In [ ]:
t0_tot = time.time()

wavelength = sc.Quantity(np.array(550), 'nm') 
theta_range = sc.Quantity(np.arange(125., 150, 2),'degrees')

refl_events = np.zeros((theta_range.size, 2*nevents+1))
refl_events_fresnel_samp = np.zeros((theta_range.size, 2*nevents+1))
refl_events_fresnel_avg = np.zeros((theta_range.size, 2*nevents+1))
reflectance = np.zeros(theta_range.size)

i = np.where(wavelengths.magnitude == wavelength.magnitude)[0]
n_sample = ri.n_eff(n_particle[i], n_matrix[i], volume_fraction)

# Calculate the phase function and scattering and absorption coefficients from the single scattering model
# (this absorption coefficient is of the scatterer, not of an absorber added to the system)
p, mu_scat, mu_abs = mc.calc_scat(particle_radius, n_particle[i], n_sample, volume_fraction, wavelengths[i])
lscat = 1/mu_scat.magnitude # microns

# Initialize the trajectories
r0, k0, W0 = mc.initialize(nevents, ntrajectories, n_medium[i], n_sample, incidence_angle = 0., seed = seed)
r0 = sc.Quantity(r0, 'um')
k0 = sc.Quantity(k0, '')
W0 = sc.Quantity(W0, '')

# Create step size distribution
step = mc.sample_step(nevents, ntrajectories, mu_abs, mu_scat)


for j in range(theta_range.size):
    t0 = time.time()
    print(theta_range[j])
    
    # Generate a matrix of all the randomly sampled angles first 
    _, _, sinphi, cosphi, _, _ = mc.sample_angles(nevents, ntrajectories, p)
    theta = np.ones(theta.shape)*theta_range[j].to('rad').magnitude
    sintheta = np.sin(theta)
    costheta = np.cos(theta)
    
    # Create trajectories object
    trajectories = mc.Trajectory(r0, k0, W0)

    # Run photons
    trajectories.absorb(mu_abs, step)    

    trajectories.scatter(sintheta, costheta, sinphi, cosphi)   
    trajectories.move(step)

    ################### Calculate reflection and transmition   
    refl_indices, trans_indices,\
    inc_refl_per_traj, refl_per_traj, trans_per_traj,\
    trans_frac, refl_frac,\
    refl_fresnel, trans_fresnel, reflectance[j] = mc.calc_refl_trans(trajectories, z_low, thickness, n_medium[i], 
                                                                     n_sample, return_extra = True)


    ################### Calculate event distribution ##########################
    
    refl_events[j,:], trans_events = ed.calc_refl_trans_event(refl_per_traj, inc_refl_per_traj, trans_per_traj, 
                              refl_indices, trans_indices, nevents)
 
    # scattering angle
    #theta_event_traj[j,:,:] = ed.calc_thetas_event_traj(theta, refl_indices, nevents, ntraj = ntraj_save)

    # fresnel
#    pdf_refl, pdf_trans =  ed.calc_pdf_scat(refl_events[i,:], trans_events, nevents)
    
#    refl_events_fresnel_avg[j,:] = ed.calc_refl_event_fresnel_avg(refl_events[i,:], refl_indices, trans_indices, 
#                            refl_fresnel, trans_fresnel,
#                            refl_frac, trans_frac, nevents)
    
    
#    refl_events_fresnel_samp[j,:] = ed.calc_refl_event_fresnel_pdf(refl_events[i,:], pdf_refl, pdf_trans, refl_indices, 
#                            refl_fresnel, trans_fresnel,
#                            refl_frac, trans_frac, nevents)

    # information about trajectories we want to save
#    exit_indices = refl_indices + trans_indices
#    for tr in range(0,ntraj_save): # loop thru all trajectories
#        zpos[j, 0:int(exit_indices[tr])+1,tr] = trajectories.position[2, 0:int(exit_indices[tr]) + 1, tr]
#        kz[j, 0:int(exit_indices[tr]),tr] = trajectories.direction[2,0:int(exit_indices[tr]), tr]
    
    # keep track of time to run
    t1 = time.time()
    t_wl = t1-t0
    print('time: ' + str(t_wl/60) + ' min')
    
t_tot = time.time()-t0_tot
print('total time: ' + str(t_tot/3600) + ' hr')

##### Save results

In [ ]:
ed.save_data(particle,
             matrix,
             particle_radius,
             volume_fraction,
             thickness,
             reflectance,
             refl_events,
             wavelength, 
             nevents, 
             ntrajectories,
             theta_range = theta_range)